In [ ]:
!pip3 install duckdb==0.3.1

### About DuckDB 
- [Official Webpage](https://duckdb.org/) 
- [Query Parquet with DuckDB](https://duckdb.org/2021/06/25/querying-parquet.html)

In [2]:
# Import necessary libraries
import pandas as pd  
import numpy as np   
import duckdb        # Import the DuckDB library for database operations
import time          

# Establish a connection to an in-memory DuckDB database
con = duckdb.connect(database=':memory:', read_only=False)

---

## Exploration Query

In [3]:
%%time  # Measure the execution time of the following code

# Execute a SQL query to count events by type in a Parquet file
result = con.query('''
SELECT event_type,
       count(*) as event_count
FROM '../Data/optimised_raw_data.parquet'
GROUP BY 1
''').df()

# Print the result
print(result)


CPU times: user 1.16 s, sys: 10.3 ms, total: 1.17 s
Wall time: 1.17 s


,event_type,event_count
0,view,52490785
1,cart,2641249
2,purchase,835007


In [4]:
# Execute a SQL query to select data from a Parquet file where the event_type is 'view'and limit the result to the first 5 rows.

result = con.query('''
SELECT *
FROM '../Data/optimised_raw_data.parquet'
WHERE event_type = 'view'
LIMIT 5
''').df()

result


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2020-01-01 00:00:00 UTC,view,1005073,2232732093077520756,construction.tools.light,samsung,1130.020020,519698804,69b5d72f-fd6e-4fed-aa23-1286b2ca89a0
1,2020-01-01 00:00:01 UTC,view,1005192,2232732093077520756,construction.tools.light,meizu,205.669998,527767423,7f596032-ccbf-4643-9bad-e36a209512b4
2,2020-01-01 00:00:01 UTC,view,100063693,2053013552427434207,apparel.shirt,turtle,136.429993,519046195,d1e2f343-84bb-49bd-b13d-ca0f1ed9910e
3,2020-01-01 00:00:01 UTC,view,5100816,2232732103831716449,apparel.shoes,xiaomi,29.950001,518269232,0444841c-38ef-410c-b11f-7b35ea4e5991
4,2020-01-01 00:00:02 UTC,view,100014325,2232732103294845523,apparel.shoes.step_ins,intel,167.199997,587748686,31b7d4cf-dfac-4895-9927-90fa3254f860


In [5]:
# Measure the execution time of the following code using %%time
%%time

# Execute a SQL query to calculate counts from a Parquet file where the event_type is 'view'

result = con.query('''
SELECT 
    COUNT(distinct product_id) as product_cnt_30days,
    COUNT(distinct user_id) as user_cnt_30days,
    COUNT(distinct user_session) as user_session_cnt_30days
FROM '../Data/optimised_raw_data.parquet' 
WHERE event_type='view'
''').df()

result


CPU times: user 18.8 s, sys: 1.22 s, total: 20 s
Wall time: 20.4 s


,product_cnt_30days,user_cnt_30days,user_session_cnt_30days
0,226209,4383932,13757026


---

### Data Null Check on user_session

In [6]:
# Execute a SQL query to count the number of rows with NULL values in specific columns

result = con.query('''
SELECT COUNT(*) as null_cnt
FROM '../Data/optimised_raw_data.parquet'
WHERE event_type = 'view'
  AND (user_session IS NULL
       OR user_id IS NULL
       OR product_id IS NULL)
''').df()

result


,null_cnt
0,0


---

### Check how many distinct PDP view event is performed per session?

In [7]:
# Execute a SQL query to count the unique product views per user session

result = con.query('''
SELECT 
    user_session,
    COUNT(distinct product_id) as count_unique_product_views_per_session
FROM '../Data/optimised_raw_data.parquet' 
WHERE event_type='view' 
GROUP BY 1
''').df()

result


,user_session,count_unique_product_views_per_session
0,69b5d72f-fd6e-4fed-aa23-1286b2ca89a0,8
1,7f596032-ccbf-4643-9bad-e36a209512b4,4
2,d1e2f343-84bb-49bd-b13d-ca0f1ed9910e,1
3,0444841c-38ef-410c-b11f-7b35ea4e5991,1
4,31b7d4cf-dfac-4895-9927-90fa3254f860,2
...,...,...
13757021,f467f7ab-6709-4836-80ba-0d1bd8d1a267,3
13757022,b73ccf13-375c-4709-a0f3-d25cc6a11ae2,1
13757023,5190c16b-f4f4-474c-ac75-de8ae413d82c,1
13757024,7c8b6690-116e-4a3f-b573-be4f981393a8,1


In [8]:
# Execute a SQL query to calculate the average count of unique product views per user session

result = con.query('''
SELECT AVG(count_unique_product_views_per_session) as avg_unique_product_views_per_session
FROM (SELECT 
    user_session,
    COUNT(distinct product_id) as count_unique_product_views_per_session
    FROM '../Data/optimised_raw_data.parquet' 
    WHERE event_type='view' 
    GROUP BY 1 ) as processed_table
''').df()

result


,avg_unique_product_views_per_session
0,2.622409


- In one user session on an average users visit 2 to 3 distinct PDPs.

---

### Count total events with non-null sessions

In [9]:
# Execute a SQL query to count the number of rows with 'event_type' equal to 'view' and 'user_session' is not NULL

result = con.query('''
SELECT COUNT(*) AS view_cnt
FROM '../Data/optimised_raw_data.parquet'
WHERE event_type = 'view'
  AND user_session IS NOT NULL
''').df()

result



,view_cnt
0,52490785


---

### EDA on user_sessions

In [10]:
# Execute a SQL query to count the distinct user sessions where 'event_type' is 'view'

result = con.query('''
SELECT count(DISTINCT user_session) as user_session_cnt
FROM '../Data/optimised_raw_data.parquet' 
WHERE event_type = 'view'
''').df()

result


,user_session_cnt
0,13757026


In [11]:
# Execute a SQL query to count the unique product views per user session

result = con.query('''
SELECT 
    user_session,
    COUNT(distinct product_id) as count_unique_product_views_per_session
FROM '../Data/optimised_raw_data.parquet' 
WHERE event_type='view' 
GROUP BY 1
''').df()

result


,user_session,count_unique_product_views_per_session
0,69b5d72f-fd6e-4fed-aa23-1286b2ca89a0,8
1,7f596032-ccbf-4643-9bad-e36a209512b4,4
2,d1e2f343-84bb-49bd-b13d-ca0f1ed9910e,1
3,0444841c-38ef-410c-b11f-7b35ea4e5991,1
4,31b7d4cf-dfac-4895-9927-90fa3254f860,2
...,...,...
13757021,f467f7ab-6709-4836-80ba-0d1bd8d1a267,3
13757022,b73ccf13-375c-4709-a0f3-d25cc6a11ae2,1
13757023,5190c16b-f4f4-474c-ac75-de8ae413d82c,1
13757024,7c8b6690-116e-4a3f-b573-be4f981393a8,1


In [12]:
# Execute a SQL query to count the unique product views per user session where the count is greater than 1

result = con.query('''
SELECT 
    user_session,
    COUNT(distinct product_id) as count_unique_product_views_per_session
FROM '../Data/optimised_raw_data.parquet' 
WHERE event_type='view' 
GROUP BY 1
HAVING COUNT(distinct product_id) > 1
''').df()

result


,user_session,count_unique_product_views_per_session
0,69b5d72f-fd6e-4fed-aa23-1286b2ca89a0,8
1,7f596032-ccbf-4643-9bad-e36a209512b4,4
2,31b7d4cf-dfac-4895-9927-90fa3254f860,2
3,c161400e-630b-4b59-8797-d9b9714444bf,2
4,44ca9bc7-3ba8-4546-bdf5-10b2172ed883,2
...,...,...
5340110,3df590c0-e667-4f5a-aa87-f47749cd6408,2
5340111,e1252736-1d34-40c5-8aab-af2e5bff3201,2
5340112,21f46782-fb36-4d5c-82d4-6dc26ae93fad,2
5340113,4d614d05-3b0c-4504-a75e-9e09715f737f,3


---

## Data Inferences

- Number of entries for `event_type = 'view'`: Over 52 million.
- On average, users on Flipkart visit approximately 2.6 (rounded to 3) distinct Product Detail Pages (PDPs) per session.
- In the month of January 2020, more than 13 million sessions occurred on Flipkart where users visited at least one PDP.
- Over 5 million sessions had more than one distinct PDP view.

---